In [4]:
import os
import sys
from pathlib import Path

# We are including our repository bin to the system path so that we can use some handy functions
bin_path = Path.cwd().parents[0] / 'bin'
if bin_path not in sys.path:
    sys.path.append(str(bin_path))

import gnssrefl_helpers

# Making sure environment variables are set - this is required to run the gnssrefl code

exists = gnssrefl_helpers.check_environment()
if exists == False:
    #If you are running this locally - make sure the items in the exe folder have execution permissions
    gnssrefl_helpers.set_environment(refl_code=str(Path.cwd().parents[0]),
                                     orbits=str(Path.cwd().parents[0] / "orbits"),
                                     exe=str(Path.cwd().parents[0] / "bin" / "exe"))

# Set local variable of refl_code location
refl_code_loc = os.environ['REFL_CODE']

environment variable ORBITS set to path /Users/enloe/UNAVCO_Projects/git/gnssrefl_jupyter/orbits 
environment variable REFL_CODE set to path /Users/enloe/UNAVCO_Projects/git/gnssrefl_jupyter 
environment variable EXE set to path /Users/enloe/UNAVCO_Projects/git/gnssrefl_jupyter/bin/exe


### Step 3. Download and check EXE dependencies are present:

use environment.download_crx2rnx to import the crx2rnx file(Required translator for compressed (Hatanaka) RINEX files) which is dependant on your working OS - this is required to run the gnssrefl code.

If this does not properly find your running os, then it will print out an error and instruct you how to add a parameter to manually set which os you are using.

Note that this function relies on your environment variables to be properly set.

In [13]:
print('files in exe folder:', os.listdir(os.environ['EXE']))

#import gnssrefl rinex2snr function (environment variables must be set first)
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.installexe_cl import installexe

# Download the executablines
try:
    os.environ['DOCKER']
except KeyError:
    sys = gnssrefl_helpers.get_sys()
    installexe(sys)

files in exe folder: ['.DS_Store', 'CRX2RNX', 'RNXCMP_4.1.0_MacOSX10.14_gcc', 'gfzrnx_2.0-8219_osx_arm64', 'teqc', 'gfzrnx_1.15-8044_lx64', 'gfzrnx_2.0-8219_armlx64', 'gfzrnx_2.0-8219_lx64', 'RNXCMP_4.1.0_MacOSX10.14_gcc.tar.gz']
Your executable environment area:  /Users/enloe/UNAVCO_Projects/git/gnssrefl_jupyter/bin/exe
You already have this executable: CRX2RNX
Executable already exists: teqc

 Executable stored: gfzrnx


If you see 'CRX2RNX' and 'gfzrnx' in your EXE folder then you are all set. 

**Note*** that the gfzrnx file was in the exe when you pulled the repository - it currently is set for a linux environment and can only be used with the docker version of the jupyter notebooks or if you are running linux. The gfzrnx file is not required to run the code - but is needed if you want to work with RINEX3 files. If you need to download the correct version for your os then download from [here](http://dx.doi.org/10.5880/GFZ.1.1.2016.002) and then place it in your exe folder.

### Step 4. Run a quick Analysis

#### a. simple use case that requires CRX2RNX and broadcast orbits:

In [6]:
station = 'p042'
year = 2018
doy = 150

To understand what rinex2snr returns, lets look at the function's available and default parameters:

In [7]:
rinex2snr?

Now lets run the function without changing any of the defaults.

In [8]:
rinex2snr(station, year, doy)

Will seek the RINEX file from an external archive
p042  year: 2018  doy: 150 from:  all
Requested a GPS only nav file
Nav file exists online
Orbit file:  /Users/enloe/UNAVCO_Projects/git/gnssrefl_jupyter/orbits/2018/nav/auto1500.18n
Searching the  sopac  archive for  p042
ftp://garner.ucsd.edu/pub/rinex/2018/150/p0421500.18d.Z

 File has been found  p0421500.18d.Z
Download took  1.3  seconds


SUCCESS: SNR file was created: /Users/enloe/UNAVCO_Projects/git/gnssrefl_jupyter/2018/snr/p042/p0421500.18.snr66
That took  1.95  seconds
Feedback written to subdirectory logs


you've successfully run the rinex2snr program that:
* downloaded and uncompressed [hatanaka](https://www.unavco.org/data/gps-gnss/hatanaka/hatanaka.html) rinex for a single station (p042) for a single day (doy 150 in 2018)
* downloaded GPS broadcast orbits
* calculated azimuth and elevation for each satellite at each epoch given these orbits
* wrote this az/el, signal, time and CN0 information to a formatted snr output file
for future analysis.
Reminder, the .66 file name suffix refers to the
[elevation masking options](https://github.com/kristinemlarson/gnssrefl#iv-rinex2snr---extracting-snr-data-from-rinex-files-).

#### b. simple use case that requires CRX2RNX and SP3 orbits:

Here we will run rinex2snr for the same day, but lets change the 'orb' parameter to gnss.

In [9]:
rinex2snr(station, year, doy=doy, orb='gnss')

SNR file already exists /Users/enloe/UNAVCO_Projects/git/gnssrefl_jupyter/2018/snr/p042/p0421500.18.snr66
That took  0.0  seconds
Feedback written to subdirectory logs


Note* If you get:
*SNR file exists...*\
This is because the logic of gnssrefl checks for an snr file prior for processing - and we already processed this day earlier.
Remember this fact if you ever want to **re**-process with different orbits!
You can use the overwrite parameter to overwrite files if you want to reprocess.
Now lets try that again.

In [10]:
rinex2snr(station, year, doy=150, orb='gnss', overwrite=True)

File exists, you requested overwriting, so will delete existing file
Will seek the RINEX file from an external archive
p042  year: 2018  doy: 150 from:  all
Type 1 filename gbm20033.sp3.Z
Type 2 filename GFZ0MGXRAP_20181500000_01D_05M_ORB.SP3.gz
/gps/products/mgex/2003/ gbm20033.sp3.Z
Orbit file:  /Users/enloe/UNAVCO_Projects/git/gnssrefl_jupyter/orbits/2018/sp3/gbm20033.sp3
Searching the  sopac  archive for  p042
ftp://garner.ucsd.edu/pub/rinex/2018/150/p0421500.18d.Z

 File has been found  p0421500.18d.Z
Download took  1.04  seconds


SUCCESS: SNR file was created: /Users/enloe/UNAVCO_Projects/git/gnssrefl_jupyter/2018/snr/p042/p0421500.18.snr66
That took  3.69  seconds
Feedback written to subdirectory logs


If you get:
SUCCESS: SNR file was created: ...
you've successfully:

* downloaded and uncompressed hatanaka rinex for a single station (p042) for a single day (doy 150 in 2018)
* downloaded SP3 format GNSS orbits from the GFZ archive
* calculated azimuth and elevation for each satellite at each epoch
* wrote this az/el, signal, time and CN0 information to a formatted snr output file for future analysis.

#### c. (OPTIONAL - requires the gfzrnx executable mentioned previously ) RINEX 3 simple use case that requires gfzrnx

**If** you are interested in using RINEX version 3 data, please run this test:

note: this will fail if you do not have the correct system-dependant gfzrnx translation file. See the instructions above to get this file.

In [14]:
rinex2snr(station='onsa00swe', year=2020, doy=1, archive='cddis', orb='gnss')

Will seek the RINEX file from an external archive
ONSA00SWE  year: 2020  doy: 1 from:  cddis
Searching the  cddis  archive with rate/filetype 30 R
Filename: ONSA00SWE_R_20200010000_01D_30S_MO.crx.gz
Download took  2.6  seconds
File exists:  ONSA00SWE_R_20200010000_01D_30S_MO.crx.gz  at  cddis
remove Hatanaka compressed file
The RINEX 3 file has been downloaded. Try to make  onsa0010.20o


The file ONSA00SWE_R_20200010000_01D_30S_MO.rnx already exists. Overwrite?(n)

Look for the rinex 2.11 file here:  onsa0010.20o
gfzrnx rinex3 to rinex 2: 6.22  seconds
RINEX 2 created from v3 2020 1  Now remove RINEX 3 files and convert
Type 1 filename gbm20863.sp3.Z
Type 2 filename GFZ0MGXRAP_20200010000_01D_05M_ORB.SP3.gz
/gps/products/mgex/2086/ gbm20863.sp3.Z
/gps/products/mgex/2086/ GFZ0MGXRAP_20200010000_01D_05M_ORB.SP3.gz
Orbit file:  /Users/enloe/UNAVCO_Projects/git/gnssrefl_jupyter/orbits/2020/sp3/GFZ0MGXRAP_20200010000_01D_05M_ORB.SP3


SUCCESS: SNR file was created: /Users/enloe/UNAVCO_Projects/git/gnssrefl_jupyter/2020/snr/onsa/onsa0010.20.snr66
That took  13.15  seconds
Feedback written to subdirectory logs


If you get:
*SUCCESS: SNR file was created: ...* \
you've successfully:
* downloaded and uncompressed rinex 3 for a single station (onsa)
for a single day (doy 1 in 2020) from the cddis archive
* converted rinex 3 to rinex 2 using gfzrnx executable
* downloaded SP3 format GNSS orbits from the GFZ archive
* calculated azimuth and elevation for each satellite at each epoch
* wrote this az/el, signal, time and CN0 information to a formatted
snr output file for future analysis.